<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/*Bhav_Keras_Image_GeneratorCNNs_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

IMPORT LIBRARIES

In [0]:
import os
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import tensorflow
import pandas as pd
import itertools
from tensorflow import keras
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model, Sequential # This is the functional API
from tensorflow.keras.optimizers import RMSprop, Adam, Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.preprocessing import image

from tensorflow.keras import applications 
from keras.utils.np_utils import to_categorical 

from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math 
import sys
 

Using TensorFlow backend.


In [0]:
#CREATE MODEL

new_model = Sequential()
new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
new_model.add(Dense(2, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
new_model.layers[0].trainable = False

In [0]:
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=1)

In [0]:
#Compile the model

new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#IMAGE LABEL AND FEATURES GENERATION
 
# loading up our datasets

TRAINING_DIR = "/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/train"

VALIDATION_DIR = "/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/validation"
 

ImageDataGenerator = tensorflow.keras.preprocessing.image.ImageDataGenerator

In [0]:

image_size = 255, 255

datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #uses Resnet's preprocess input function


In [10]:

 
train_generator = datagen.flow_from_directory(TRAINING_DIR,
                                              batch_size=10,
                                              class_mode='categorical',
                                              target_size=(255, 255), 
                                              shuffle=True, 
                                              seed=42)
 

validation_generator = datagen.flow_from_directory(VALIDATION_DIR,
                                              batch_size=10,
                                              class_mode='categorical',
                                              target_size=(255, 255), 
                                              shuffle=True, 
                                              seed=42)


Found 519 images belonging to 2 classes.
Found 182 images belonging to 2 classes.


TRAIN AND VALIDATE ON DATA

In [0]:
initial_epochs = 10  #can increase to 10
#validation_steps=20

#loss0,accuracy0 = model.evaluate(validation_generator, steps = validation_steps)

#print("initial loss: {:.2f}".format(loss0))
#print("initial accuracy: {:.2f}".format(accuracy0))

In [12]:
epochs = 10

#batch_size=32

stop = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.001, patience=3, verbose=1)
filepath="/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/best_resnet_model_scenes_assignment_imagegeneratorv3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')

new_model.fit_generator(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        verbose=1,
        steps_per_epoch=2000,
        callbacks=[stop, checkpoint, rlrop], 
        validation_steps=800)




Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
2000/2000 [==============================] - ETA: 0s - loss: 0.0095 - accuracy: 0.9977
Epoch 00001: val_loss improved from inf to 0.04062, saving model to /content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/best_resnet_model_scenes_assignment_imagegeneratorv3.hdf5
2000/2000 [==============================] - 115s 58ms/step - loss: 0.0095 - accuracy: 0.9977 - val_loss: 0.0406 - val_accuracy: 0.9834 - lr: 0.0010
Epoch 2/10
1999/2000 [============================>.] - ETA: 0s - loss: 2.8028e-04 - accuracy: 1.0000
Epoch 00002: val_loss did not improve from 0.04062
2000/2000 [==============================] - 114s 57ms/step - loss: 2.8016e-04 - accuracy: 1.0000 - val_loss: 0.0447 - val_accuracy: 0.9837 - lr: 0.0010
Epoch 3/10
2000/2000 [==============================] - ETA: 0s - loss: 1.2756e-04 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.04062
2000/2000 [================